In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
# age
def age_group(x):
    if x<=20:
        return "20↓" 
    elif x>20 and x<=30:
        return "20-30" 
    elif x>30 and x<=40:
        return "30-40" 
    elif x>40 and x<=50:
        return "40-50"
    elif x>50 and x<=60:
        return "50-60" 
    elif x>60:
        return "60↑"

In [3]:
# area
def city_group(x):
    if x =="None":
        return "None"
    
    city = {
        "North":["台北市", "新北市", "基隆市", "新竹市", "桃園市", "新竹縣"],
        "Center":["台中市", "苗栗縣", "彰化縣", "南投縣", "雲林縣"],
        "South":["高雄市", "台南市", "嘉義市", "嘉義縣", "屏東縣"],
        "East":["宜蘭縣", "花蓮縣", "台東縣"],
        "offshore_islands ":["金門縣", "澎湖縣"]
    }
    
    for area in city:
        if x in city[area]:
            return area

In [4]:
# read data
df = pd.read_csv('../data/Onshore.csv')

# add the number of customers
df["customer_num"] = df['female'] + df['male']

# add recommended score
df["score"] = 0

In [5]:
df

,sales,female,male,20-30,20↓,30-40,40-50,50-60,60↑,highyielddebtriskforecast: N,...,location_新北市,location_新竹市,location_新竹縣,location_桃園市,location_花蓮縣,location_苗栗縣,location_雲林縣,location_高雄市,customer_num,score
0,311710,1,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,A000159078,2,6,2,4,0,2,0,0,8,...,5,0,0,0,0,0,0,0,8,0
2,A0278,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,A0288,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,A0367,0,3,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,3,0
5,A0431,8,4,3,2,2,1,3,1,10,...,3,1,1,1,0,0,0,1,12,0
6,A0460,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
7,A0524,11,3,2,0,3,5,3,1,11,...,3,0,1,1,0,0,1,3,14,0
8,A0532,2,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,2,0
9,A0639,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
dic = df.to_dict(orient='records')
dic

[{'sales': '311710',
  'female': 1,
  'male': 0,
  '20-30': 0,
  '20↓': 0,
  '30-40': 0,
  '40-50': 1,
  '50-60': 0,
  '60↑': 0,
  'highyielddebtriskforecast: N': 1,
  'highyielddebtriskforecast: Y': 0,
  'area_Center': 0,
  'area_East': 0,
  'area_North': 1,
  'area_South': 0,
  'location_台中市': 0,
  'location_台北市': 0,
  'location_台南市': 0,
  'location_嘉義市': 0,
  'location_嘉義縣': 0,
  'location_基隆市': 0,
  'location_屏東縣': 0,
  'location_彰化縣': 0,
  'location_新北市': 1,
  'location_新竹市': 0,
  'location_新竹縣': 0,
  'location_桃園市': 0,
  'location_花蓮縣': 0,
  'location_苗栗縣': 0,
  'location_雲林縣': 0,
  'location_高雄市': 0,
  'customer_num': 1,
  'score': 0},
 {'sales': 'A000159078',
  'female': 2,
  'male': 6,
  '20-30': 2,
  '20↓': 4,
  '30-40': 0,
  '40-50': 2,
  '50-60': 0,
  '60↑': 0,
  'highyielddebtriskforecast: N': 8,
  'highyielddebtriskforecast: Y': 0,
  'area_Center': 0,
  'area_East': 0,
  'area_North': 8,
  'area_South': 0,
  'location_台中市': 0,
  'location_台北市': 3,
  'location_台南市': 0,
  '

In [7]:
#age/area/location/gender/risk
age = int(input("age:" ))
gender = input("female/male: ")
city = input("city: ")

age_range = age_group(age)
area = "area_" + city_group(city)
location = "location_" + city
applicant = {"age": age_range, "gender": gender, "area": area, "location": location}

age:35
female/male: male
city: 新竹縣


In [8]:
def recommend(applicant):
    
    print("---------filter by area-----------")
    data_filter_area = []
    for data in dic:
        if data[applicant["area"]] > 0:
            print(data["sales"])
            data_filter_area.append(data)
    
    print("---------filter by age-----------")
    data_filter_age = []
    for data in data_filter_area:
        if data[applicant["age"]] > 0:
            data["score"] += data[applicant["age"]] / data["customer_num"]
            print(data["sales"], data["score"])
            data_filter_age.append(data)
            
    print("---------filter by gender-----------")
    data_filter_gender = []
    for data in data_filter_age:
        if data[applicant["gender"]] > 0:
            data["score"] += (data[applicant["gender"]] / data["customer_num"] * 0.8)
            print(data["sales"], data["score"])
            data_filter_gender.append(data)
            
    print("---------filter by location-----------")
    data_filter_loc = []
    for data in data_filter_age:
        if data[applicant["location"]] > 0:
            data["score"] += (data[applicant["location"]] / data["customer_num"] * 0.6)
            print(data["sales"], data["score"])
            data_filter_loc.append(data)
            
    if len(data_filter_loc) > 0:
        return get_best_sales(data_filter_loc)
    elif len(data_filter_gender) > 0:
        return get_best_sales(data_filter_gender)
    elif len(data_filter_age) > 0:
        return get_best_sales(data_filter_age)
    else:
        return get_best_sales(data_filter_area)
        
def get_best_sales(sales):
    
    best_score = 0
    best_sales_index = 0
    for i in range(len(sales)):
        if sales[i]["score"] > best_score:
            best_score = sales[i]["score"]
            best_sales_index = i
    
    return sales[best_sales_index]["sales"]
        
        
    

In [9]:
best_one = recommend(applicant)

---------filter by area-----------
311710
A000159078
A0278
A0431
A0460
A0524
A0532
A0700
A0769
A0780
C000467580
H000141315
J000725232
L000523853
LF00000001
RCAM1
---------filter by age-----------
A0431 0.16666666666666666
A0460 1.0
A0524 0.21428571428571427
A0532 0.5
A0769 0.2916666666666667
C000467580 1.0
J000725232 1.0
RCAM1 0.125
---------filter by gender-----------
A0431 0.43333333333333335
A0524 0.3857142857142857
A0769 0.5916666666666668
RCAM1 0.525
---------filter by location-----------
A0431 0.48333333333333334
A0524 0.42857142857142855
A0769 0.6166666666666668


In [10]:
best_one

'A0769'